In [2]:
import tensorflow as tf
from tensorflow.keras import Input, layers, Model
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
import numpy as np
import matplotlib.pyplot as plt

### Creating Mask

In [6]:
mask = np.load('../data/mask_global.npy')
mask = tf.keras.backend.constant(mask)
mask = tf.expand_dims(mask, -1)

### Training 4 Hetero-ConvLSTMS

In [7]:
lookback = 7
batch_size = 4

train_X_crimes_only = np.load('../data/train_data.npy')
test_X_crimes_only = np.load('../data/test_data.npy')

train_X_crimes_only = tf.expand_dims(train_X_crimes_only, -1)
test_X_crimes_only = tf.expand_dims(test_X_crimes_only, -1)

train_gen = TimeseriesGenerator(
    train_X_crimes_only,
    train_X_crimes_only,
    length=lookback,
    batch_size=batch_size,
    shuffle=False
)
test_gen = TimeseriesGenerator(
    test_X_crimes_only,
    test_X_crimes_only,
    length=lookback,
    batch_size=batch_size,
    shuffle=False
)

def masked_MSE_loss(y_true, y_pred):
    y_pred_masked = tf.math.multiply(y_pred, mask)
    mse = tf.keras.losses.mean_squared_error(y_true = y_true, y_pred = y_pred_masked)
    return mse

# Define the input tensors
inputs = Input(shape=(lookback, *train_X_crimes_only.shape[1:]))

# First stack of convlstm layers
convlstm1 = layers.ConvLSTM2D(filters=128, kernel_size=(3, 3), padding='same', activation='tanh', return_sequences=True)(inputs)
bathnorm1 = layers.BatchNormalization()(convlstm1)
convlstm2 = layers.ConvLSTM2D(filters=128, kernel_size=(3, 3), padding='same', activation='tanh', return_sequences=False)(bathnorm1)

# Second stack of convlstm layers
convlstm3 = layers.ConvLSTM2D(filters=128, kernel_size=(3, 3), padding='same', activation='tanh', return_sequences=True)(inputs)
batchnorm2 = layers.BatchNormalization()(convlstm3)
convlstm4 = layers.ConvLSTM2D(filters=128, kernel_size=(3, 3), padding='same', activation='tanh', return_sequences=False)(batchnorm2)

# Concatenate outputs of two stacks
concatenation = layers.concatenate([convlstm2, convlstm4])
outputs = layers.Conv2D(filters=1, kernel_size=1, padding="same", activation='linear')(concatenation)

# Create the model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss=masked_MSE_loss, metrics=['mae'])

# Train the model
model.fit(train_gen, epochs=1)

# Create test prediction
test_pred = model.predict(test_gen)
test_pred *= mask

90/90 [==============================] - 14s 145ms/step


In [11]:
np.save('../data/homo_convlstm.npy', test_pred)